In [1]:
!unzip /home/aistudio/data/data78992/lcqmc.zip -d /home/aistudio/data/
# !unzip /home/aistudio/data/data78992/paws-x-zh.zip -d /home/aistudio/data/
# !unzip /home/aistudio/data/data78992/bq_corpus.zip -d /home/aistudio/data/

Archive:  /home/aistudio/data/data78992/lcqmc.zip


   creating: /home/aistudio/data/lcqmc/
  inflating: /home/aistudio/data/lcqmc/train.tsv  


   creating: /home/aistudio/data/__MACOSX/
   creating: /home/aistudio/data/__MACOSX/lcqmc/
  inflating: /home/aistudio/data/__MACOSX/lcqmc/._train.tsv  
  inflating: /home/aistudio/data/lcqmc/dev.tsv  
  inflating: /home/aistudio/data/__MACOSX/lcqmc/._dev.tsv  
  inflating: /home/aistudio/data/lcqmc/License.pdf  
  inflating: /home/aistudio/data/__MACOSX/lcqmc/._License.pdf  
  inflating: /home/aistudio/data/lcqmc/test.tsv  
  inflating: /home/aistudio/data/__MACOSX/lcqmc/._test.tsv  
  inflating: /home/aistudio/data/lcqmc/User_Agreement.pdf  
  inflating: /home/aistudio/data/__MACOSX/lcqmc/._User_Agreement.pdf  
  inflating: /home/aistudio/data/__MACOSX/._lcqmc  


## 1. Load dataset

In [2]:
def load_dataset(fpath, num_row_skip=0):

    def read(fp):
        data = open(fp)

        for _ in range(num_row_skip):
            next(data)

        if "test" in fp:
            for line in data:
                line = line.strip().split('\t')
                yield line[0], line[1]
        else:
            for line in data:
                line = line.strip().split('\t')
                if len(line) == 3:
                    yield line[0], line[1], int(line[2])

    if isinstance(fpath, str):
        return list(read(fpath))
    elif isinstance(fpath, (list, tuple)):
        return [list(read(fp)) for fp in fpath]
    else:
        raise TypeError("Input fpath must be a str or a list/tuple of str")

In [3]:
train_set, dev_set, test_set = load_dataset(['./data/lcqmc/train.tsv', './data/lcqmc/dev.tsv', './data/lcqmc/test.tsv'])
# len(train_set), len(dev_set), len(test_set)
train_set = train_set + dev_set

## 2. Transform text

In [4]:
from paddlenlp.datasets import MapDataset
from paddle.io import BatchSampler, DataLoader
from paddlenlp.data import Pad, Stack, Tuple
from paddlenlp.transformers import RobertaModel as SeqClfModel
from paddlenlp.transformers import RobertaTokenizer as PTMTokenizer
import numpy as np


MODEL_NAME = "roberta-wwm-ext-large"
tokenizer = PTMTokenizer.from_pretrained(MODEL_NAME)


def example_converter(example, max_seq_length, tokenizer):
    text_a, text_b, label = example
    encoded = tokenizer(text=text_a, text_pair=text_b, max_seq_len=max_seq_length)
    input_ids = encoded["input_ids"]
    token_type_ids = encoded["token_type_ids"]
    label = np.array([label], dtype="int64")
    return input_ids, token_type_ids, label


def get_trans_fn(max_seq_length=128, tokenizer=tokenizer):
    return lambda ex: example_converter(ex, max_seq_length, tokenizer)


batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),
    Stack(dtype="int64")
    ): fn(samples)


def create_dataloader(dataset,
                      trans_fn,
                      batchify_fn,
                      test=False,
                      batch_size=128,
                      shuffle=True,
                      sampler=BatchSampler):

    if test:
        dataset = [d + (0,) for d in dataset]

    if not isinstance(dataset, MapDataset):
        dataset = MapDataset(dataset)

    dataset.map(trans_fn)
    batch_sampler = sampler(dataset,
                            shuffle=shuffle,
                            batch_size=batch_size)

    dataloder = DataLoader(dataset,
                           batch_sampler=batch_sampler,
                           collate_fn=batchify_fn)

    return dataloder

[2022-03-29 04:05:19,793] [    INFO]

 - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/roberta_large/vocab.txt and saved to /home/aistudio/.paddlenlp/models/roberta-wwm-ext-large

[2022-03-29 04:05:19,796] [    INFO]

 - Downloading vocab.txt from https://paddlenlp.bj.bcebos.com/models/transformers/roberta_large/vocab.txt

  0%|          | 0/107 [00:00<?, ?it/s]

100%|██████████| 107/107 [00:00<00:00, 29425.03it/s]

In [5]:
max_seq_length = 128; batch_size = 32
trans_fn = get_trans_fn(max_seq_length)
train_loader = create_dataloader(train_set, trans_fn, batchify_fn, batch_size=batch_size)
# dev_loader = create_dataloader(dev_set, trans_fn, batchify_fn, batch_size=batch_size)
test_loader = create_dataloader(test_set, trans_fn, batchify_fn, shuffle=False, test=True, batch_size=batch_size)

## 3. Model building

In [6]:
from paddle import nn
import paddle


class PTM(nn.Layer):

    def __init__(self, pretrained_model, dropout=0.1, num_class=2):
        super().__init__()

        self.ptm = pretrained_model
        ptm_out_dim = self.ptm.config["hidden_size"]
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(ptm_out_dim, ptm_out_dim // 2)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(ptm_out_dim // 2, num_class)

    def encoder(self, input_ids, token_type_ids):
        _, embd = self.ptm(input_ids, token_type_ids)
        embd = self.dropout(embd)
        return embd

    def forward(self, input_ids, token_type_ids):
        embd = self.encoder(input_ids, token_type_ids)
        hidden = self.relu(self.fc1(embd))
        logits = self.fc2(hidden)
        return logits

In [7]:
from paddlenlp.transformers import LinearDecayWithWarmup

epoch = 4
weight_decay = 0.0
warmup_proportion = 0.0
lr_scheduler = LinearDecayWithWarmup(5e-5, len(train_loader) * epoch,
                                         warmup_proportion)

def get_model(model):
    decay_params = [
        p.name for n, p in model.named_parameters()
        if not any(nd in n for nd in ["bias", "norm"])
    ]
    optimizer = paddle.optimizer.AdamW(
    parameters=model.parameters(),
    learning_rate=lr_scheduler,
    weight_decay=weight_decay,
    apply_decay_param_fun=lambda x: x in decay_params)

    criterion = paddle.nn.CrossEntropyLoss()

    model = paddle.Model(model)
    metric = paddle.metric.Accuracy()
    model.prepare(optimizer, criterion, metric)
    return model

In [8]:
ptm = SeqClfModel.from_pretrained(MODEL_NAME)
model = PTM(ptm)
model = get_model(model)

[2022-03-29 04:05:19,960] [    INFO]

 - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/roberta_large/roberta_chn_large.pdparams and saved to /home/aistudio/.paddlenlp/models/roberta-wwm-ext-large

[2022-03-29 04:05:19,963] [    INFO]

 - Downloading roberta_chn_large.pdparams from https://paddlenlp.bj.bcebos.com/models/transformers/roberta_large/roberta_chn_large.pdparams

  0%|          | 0/1271615 [00:00<?, ?it/s]

...

 99%|█████████▊| 1254147/1271615 [00:18<00:00, 72837.99it/s]

 99%|█████████▉| 1261635/1271615 [00:18<00:00, 73438.00it/s]

100%|█████████▉| 1269149/1271615 [00:18<00:00, 73940.42it/s]

100%|██████████| 1271615/1271615 [00:18<00:00, 69365.70it/s]

W0329 04:05:38.367060   255 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0329 04:05:38.372881   255 device_context.cc:465] device: 0, cuDNN Version: 7.6.


## 4. Model training

In [9]:
model.fit(train_loader, epochs=epoch, verbose=2, log_freq=100)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.

Epoch 1/4

step  100/7737 - loss: 0.1844 - acc: 0.8063 - 311ms/step


step  200/7737 - loss: 0.2941 - acc: 0.8422 - 305ms/step


step  300/7737 - loss: 0.1282 - acc: 0.8554 - 303ms/step


...

step 7400/7737 - loss: 0.0102 - acc: 0.9673 - 306ms/step


step 7500/7737 - loss: 0.0434 - acc: 0.9674 - 306ms/step


step 7600/7737 - loss: 0.0527 - acc: 0.9674 - 306ms/step


step 7700/7737 - loss: 0.0525 - acc: 0.9674 - 306ms/step


step 7737/7737 - loss: 0.0125 - acc: 0.9674 - 306ms/step


## 5. Prediction

In [10]:
import paddle.nn.functional as F


predictions = []
logits = model.predict(test_loader)

for batch in logits[0]:
    batch = paddle.to_tensor(batch)
    probs = F.softmax(batch, axis=1)
    preds = paddle.argmax(probs, axis=1).numpy().tolist()
    predictions.extend(preds)

Predict begin...

step   2/391 [..............................]

 - ETA: 33s - 85ms/step



step   4/391 [..............................]

...

 - ETA: 0s - 74ms/step



step 391/391 [==============================]

 - 74ms/step          


Predict samples: 12500

In [11]:
with open('lcqmc.tsv', 'w') as f:
    f.write("index\tprediction")
    for idx, p in enumerate(predictions):
        f.write(f"\n{idx}\t{p}")
    f.close()